In [1]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout
from tensorflow import keras
import pandas as pd
import numpy as np
import random
import sys
import io
import os

In [2]:
df = pd.read_csv('brand_name.csv')

In [4]:
# df.head()

In [5]:
df_new = df[df['Molecules name'] == 'DABIGATRAN']

In [6]:
df_new

,Country,Drug name,Molecules name,Therapeutic 1,Therapeutic 2


In [7]:

names = (df['Country'].value_counts())
names
# for i in names:
#     print(i)

GERMANY     16159
US          12717
JAPAN        9068
VIETNAM      7901
INDIA        7796
            ...  
ALGERIA       685
SLOVENIA      599
BOSNIA        500
KUWAIT        464
SERBIA        448
Name: Country, Length: 75, dtype: int64

In [8]:
df['Country'].value_counts()

GERMANY     16159
US          12717
JAPAN        9068
VIETNAM      7901
INDIA        7796
            ...  
ALGERIA       685
SLOVENIA      599
BOSNIA        500
KUWAIT        464
SERBIA        448
Name: Country, Length: 75, dtype: int64

In [9]:
df['Therapeutic 1'].value_counts()

N2 ANALGESICS                  63374
M1 ANTIRHEUMATIC SYSTEM        42002
B1 ANTITHROMBOTIC AGENTS       22584
M5 OTHER MUSCULO-SKELETAL      17758
N1 ANAESTHETICS                15245
M2 ANTIRHEUMATICS TOPICAL      15239
L4 IMMUNOSUPPRESSANTS          10225
M3 MUSCLE RELAXANTS             9816
A1 STOMATOLOGICALS              9297
D3 WOUNDHEALING AGENTS          7530
L3 IMMUNOSTIMULATING AGENTS     4818
M4 ANTI-GOUT PREPARATIONS       3628
Name: Therapeutic 1, dtype: int64

In [10]:
df = df.drop_duplicates()

In [11]:
df_train_2 = df['Drug name']

In [12]:
text = df_train_2

In [13]:
df_train = df[df['Country'] == 'INDIA']['Drug name']

In [14]:
df_train

76130      DOMERIC
76131       HYDENT
76132     HYDENT-K
76133     KENACORT
76135       ORAHEX
           ...    
83916      NIGRAIN
83919    NOMIGRAIN
83920       GRATON
83922       MIGROF
83924       NAXDOM
Name: Drug name, Length: 5103, dtype: object

In [15]:
df_train = df_train.drop_duplicates()

In [16]:
df_train

76130      DOMERIC
76131       HYDENT
76132     HYDENT-K
76133     KENACORT
76135       ORAHEX
           ...    
83916      NIGRAIN
83919    NOMIGRAIN
83920       GRATON
83922       MIGROF
83924       NAXDOM
Name: Drug name, Length: 4642, dtype: object

In [17]:
text = df_train

In [18]:
# Data loading 
# path = 'data.txt'
# with io.open(path, encoding='utf-8') as f:
#     text = f.read().lower()

In [19]:
# Check length 
print('length:', len(text))
# Output:
# length: 1908035

length: 4642


In [20]:
# C all the characters present 
chars = sorted(list(set(text)))
print('total chars:', len(chars))
print(chars)

total chars: 4642
['3D', 'A STAR', 'A STAR-P', 'AARTHER-SP', 'AATHER P', 'ABCIXIREL', 'ABUSTAT', 'AC', 'AC PC', 'AC-1', 'AC-MR', 'AC-PARA', 'AC-SERA', 'ACCECLOWOC-MR', 'ACCET-MR', 'ACCET-P', 'ACCET-SP', 'ACCETIZ-P', 'ACE +', 'ACE VAH-PS', 'ACE XEROFLAR', 'ACE-AXOFEN', 'ACE-PROXYVON', 'ACE-PROXYVON CR', 'ACE-PROXYVON TH', 'ACE-PROXYVON-TOP.', 'ACE-RABAZIO', 'ACEBID', 'ACEBID-PLUS', 'ACECLAN', 'ACECLAN PLUS', 'ACECLAN-TH', 'ACECLO', 'ACECLO PLUS', 'ACECLO-MR', 'ACECLO-SERA', 'ACECLOFINE-P', 'ACECLOKOP PLUS', 'ACECLOMOL', 'ACECLONIJ', 'ACECLONIJ-P', 'ACECLOREN', 'ACECOFEN-P', 'ACEFEN', 'ACEFEN P', 'ACEFEN SP', 'ACEFLAM', 'ACEFORCE-MR', 'ACEFORCE-P', 'ACEFORCE-SP', 'ACEFORD-MAX', 'ACEFORD-P', 'ACEGABA', 'ACEGESIC', 'ACEJET-P', 'ACEKAY', 'ACEKAY PLUS', 'ACEKAY SERA', 'ACEKON P', 'ACELIFE-MR', 'ACELIFE-PLUS', 'ACEMARC', 'ACEMARC-MR', 'ACEMARC-PLUS', 'ACEMARC-RB', 'ACEMARC-SP', 'ACEMARC-T', 'ACEMED', 'ACEMED PLUS', 'ACEMED SP', 'ACEMED-MR', 'ACEMIZ', 'ACEMOL WESC', 'ACEMOVE-MR', 'ACEN', 'ACEN

In [21]:
# Get rid of useless shit 
text = text.replace("'", "")

In [22]:
chars = ['\n', "'", 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ','_','-','.','/','&','1','2','3','4','5','6','7','8','9','0','+','(',')','>','<','%','`']

In [23]:
# Set indices, to construct a word 
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print(char_indices)

{'\n': 0, "'": 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'x': 25, 'y': 26, 'z': 27, ' ': 28, '_': 29, '-': 30, '.': 31, '/': 32, '&': 33, '1': 34, '2': 35, '3': 36, '4': 37, '5': 38, '6': 39, '7': 40, '8': 41, '9': 42, '0': 43, '+': 44, '(': 45, ')': 46, '>': 47, '<': 48, '%': 49, '`': 50}


In [24]:
col_one_list = text.tolist()

In [25]:
col_one_list = [i.lower() for i in col_one_list]
col_one_list = [i.split(" ") for i in col_one_list]
col_one_list = [i[0] for i in col_one_list]
col_one_list = set(col_one_list)

In [26]:
col_one_list

{'aceparon',
 'diclomax',
 'clofen',
 'acho',
 'aldid',
 'aceforce-mr',
 'neupeg',
 'glenstim',
 'diclace-p',
 'hyfer',
 'onalide',
 'nimetic-mr',
 'paralaps',
 'dicsam-sp',
 'ropin',
 'bishgarbh',
 'butodol',
 'ketoflam-t',
 'arnopen',
 'frenac-mr',
 'nunock',
 'stanac-p',
 'wh5',
 'zix-mr',
 'resinim-sp',
 'solutin',
 'nudol-p',
 'oxydin',
 'ecocin',
 'clofenac',
 'alfenac-sp',
 'jusgo',
 'dolokind-aa',
 'tizan',
 'clopilar',
 'tripcy-d',
 'lupident',
 'rinhepa',
 'rinim-p',
 'afdifen',
 'dorfree',
 'stroncip',
 'glucosam',
 'wellbone',
 'effect',
 'anabel-ir',
 'acuflam',
 'zamadol',
 'heal-o-kind',
 'vecubro',
 'cenorol',
 'rimapurinol',
 'd.d.',
 'xymepill',
 'vedaz-dgm',
 'osteocerin',
 'zorrent',
 'retopyrin',
 'wintogeno',
 'omnident',
 'etori',
 'diclomac',
 'dicloflam',
 'bandrone',
 'dmlef',
 'fivasa',
 'fifanac-sp',
 'prestiflam',
 'speeder-plus',
 'tolflex',
 'larinox',
 'benzofen',
 'rheumartho-gold',
 'tixy',
 'essmol-af',
 'rightflam',
 'lorcam',
 'labdic',
 'espidase-d

In [27]:
# Split into lines and get rid of empty lines 
lines = col_one_list
lines = [line for line in lines if len(line)!=0]
print("number of lines:", len(lines))

number of lines: 4182


In [28]:
# Get length of longest word
maxlen = len(max(lines, key=len)) + 15
minlen = len(min(lines, key=len))

print("line with longest length: "+ str(maxlen))
print("line with shorter length: "+ str(minlen))

line with longest length: 33
line with shorter length: 1


In [29]:
steps = 1
sequences = []
next_chars = []

for line in lines:
    # pre-padding with zeros
    s = (maxlen - len(line))*'0' + line
    sequences.append(s)
    next_chars.append('\n')
    for it,j in enumerate(line):
        if (it >= len(line)-1):
            continue
        s = (maxlen - len(line[:-1-it]))*'0' + line[:-1-it]
        sequences.append(s)
        next_chars.append(line[-1-it])

In [30]:
print('total sequences:', len(sequences))
print(sequences[66], next_chars[66])
print(sequences[67], next_chars[67])
print(sequences[68], next_chars[68])

total sequences: 32072
00000000000000000000000000000hyfe r
000000000000000000000000000000hyf e
0000000000000000000000000000000hy f


In [31]:
# Vectorization
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)
for i, seq in enumerate(sequences):
    for t, char in enumerate(seq):
        if char != '0':
            x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

<ipython-input-31-2b5202f198e6>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
<ipython-input-31-2b5202f198e6>:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sequences), len(chars)), dtype=np.bool)


In [32]:
prefix = ""
max_names = 10

def sample(preds):
    """ function that sample an index from a probability array """
    preds = np.asarray(preds).astype('float64')
    preds = preds / np.sum(preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.random.choice(range(len(chars)), p = probas.ravel())

def print_name_generated(name):
    print(name, flush=True)
def print_list_generated(lst):
    print(lst, flush=True)
    
    
def generate_new_names(*args):
    print("----------Generatinig names----------")

    # Add pre-padding of zeros in the input.
    sequence = ('{0:0>' + str(maxlen) + '}').format(prefix).lower()

    # tmp variables
    tmp_generated = prefix
    list_outputs = list()

    while (len(list_outputs) < max_names):

        # Vectorize the input of the model.
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sequence):
            if char != '0':
                x_pred[0, t, char_indices[char]] = 1

        # Predict the probabilities of the next char.
        preds = model.predict(x_pred, verbose=0)[0]

        # Chose one based on the distribution obtained in the output of the model.
        next_index = sample(preds)
        # Get the corresponding char.
        next_char = indices_char[next_index]

        # If the char is a new line character or the name start to be bigger than the longest word, 
        # try to add it to the list and reset temp variables.
        if next_char == '\n' or len(tmp_generated) > maxlen:
            
            # If the name generated is not in the list, append it and print it.
            if tmp_generated not in list_outputs:
                list_outputs.append(tmp_generated)
                print_name_generated(tmp_generated)
            # Reset tmp variables
            sequence = ('{0:0>' + str(maxlen) + '}').format(prefix).lower()
            tmp_generated = prefix
        else:
    
            # Append the char to the sequence that we're generating.
            tmp_generated += next_char
            # Add pre-padding of zeros to the sequence generated and continue.
            sequence = ('{0:0>' + str(maxlen) + '}').format(tmp_generated).lower()
            
    # Show the intersection of the words generated and your dataset. . 
    print("Set of words already in the dataset:")
    print_list_generated(set(lines).intersection(list_outputs))
    
    # Show the rate of how many repeated words you've created.
    total_repited = len(set(lines).intersection(list_outputs))
    total = len(list_outputs)
    print("Rate of total invented words: " + "{:.2f}".format((total-total_repited)/total))
    print("-----------------End-----------------")
    
# Function invoked at the end of each epoch. Prints generated names.
callback = LambdaCallback(on_epoch_end=generate_new_names)

In [33]:
# build and train the model
# model = Sequential()
# model.add(LSTM(64, input_shape=(maxlen, len(chars))))
# model.add(Dense(len(chars), activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))
# history = model.fit(x, y, batch_size=128, epochs=2, verbose=2, callbacks=[callback])

In [34]:
# Serbia took 20- epochs to give good results, least data points 
# India took 10 epochs to give good results, max data points 



In [35]:
# model 2
model = Sequential()
model.add(LSTM(64, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.3))
# model.add(LSTM(32))
# model.add(Dropout(0.3))
model.add(Dense(len(chars), activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt)
# model.compile(loss='categorical_crossentropy', optimizer=)
history = model.fit(x, y, batch_size=128, epochs=10, verbose=2, callbacks=[callback])

Epoch 1/10
----------Generatinig names----------
auzek
isu
upanol
'robalqies
upif
ef
dlupkids
ragrrela
injoraen-sr
urfase
Set of words already in the dataset:
set()
Rate of total invented words: 1.00
-----------------End-----------------
251/251 - 12s - loss: 2.6822 - 12s/epoch - 49ms/step
Epoch 2/10
----------Generatinig names----------
enoneapon
laigcoje-d
hibume
salutirxa
ecoroceol
zomicu
etinqul-mr
tingon
al-ve
esofla-
Set of words already in the dataset:
set()
Rate of total invented words: 1.00
-----------------End-----------------
251/251 - 11s - loss: 2.3417 - 11s/epoch - 45ms/step
Epoch 3/10
----------Generatinig names----------
ai
aprogest
blopar
redbalta-ac
ebiden
olnogel
afed
ogrim-s-ra
anacol
ked-phimin
Set of words already in the dataset:
set()
Rate of total invented words: 1.00
-----------------End-----------------
251/251 - 11s - loss: 2.2526 - 11s/epoch - 43ms/step
Epoch 4/10
----------Generatinig names----------
onvonical-s
uefcclo-a-nin
upindic
edcet
ulgric-p
icfas-dp

In [36]:
# # model 5
# model = Sequential()
# model.add(LSTM(64,return_sequences=True, input_shape=(maxlen, len(chars))))
# model.add(Dropout(0.3))
# model.add(LSTM(64))
# model.add(Dropout(0.3))
# model.add(Dense(len(chars), activation='softmax'))
# opt = keras.optimizers.Adam(learning_rate=0.01)
# model.compile(loss='categorical_crossentropy', optimizer=opt)
# # model.compile(loss='categorical_crossentropy', optimizer=)
# history = model.fit(x, y, batch_size=128, epochs=10, verbose=2, callbacks=[callback])

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                29696     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 51)                3315      
                                                                 
Total params: 33,011
Trainable params: 33,011
Non-trainable params: 0
_________________________________________________________________


In [40]:
pip install keras-visualizer

Note: you may need to restart the kernel to use updated packages.


In [41]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


In [42]:
pip install pydot

Note: you may need to restart the kernel to use updated packages.


In [46]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [45]:
from keras_visualizer import visualizer 
visualizer(model, format='png', view=True)

ValueError: Keras Visualizer: Layer not supported for visualizing

In [ ]:
# model 3 
# model = Sequential()
# model.add(LSTM(128, input_shape=(maxlen, len(chars))))
# model.add(Dropout(0.3))
# model.add(Dense(len(chars), activation='softmax'))
# opt = keras.optimizers.Adam(learning_rate=0.01)
# model.compile(loss='categorical_crossentropy', optimizer=opt)
# # model.compile(loss='categorical_crossentropy', optimizer=)
# history = model.fit(x, y, batch_size=256, epochs=5, verbose=2, callbacks=[callback])

In [ ]:
# model 4
# model = Sequential()
# model.add(LSTM(64, input_shape=(maxlen, len(chars))))
# model.add(Dropout(0.3))
# model.add(Dense(len(chars), activation='softmax'))
# opt = keras.optimizers.Adam(learning_rate=0.01)
# model.compile(loss='categorical_crossentropy', optimizer=opt)
# # model.compile(loss='categorical_crossentropy', optimizer=)
# history = model.fit(x, y, batch_size=128, epochs=10, verbose=2, callbacks=[callback])

In [ ]:
# Insert a prefix of the name you'd like to generate 
# (it could be empty)
# DAB, ABI, BIG, ANT
# def generator():
#     words = ["dab","abi","big","ant"]
#     for word in words:
#         prefix = word
#         max_names = 20
#         generate_new_names()
        

# generator()
# prefix = "dab"
# prefix = "abi"
# prefix = "big"
prefix = "ant"
# # Insert how many names you'd like to generate.
max_names = 20

generate_new_names()

In [ ]:
# suffix ka badh mai i c